# Import module 

Most codes from https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/master/EBGAN.py

In [1]:
import os
import math
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='1'
mnist = input_data.read_data_sets("data/mnist",one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Helper Functions

In [3]:
def bn(net,scope,is_training):
    return tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope=scope)

def conv(net,wscope,bscope,output_depth = 64, receptive_field=[5,5],stride=[2,2]):
    shape = net.get_shape()
    weights = tf.get_variable(wscope, receptive_field+[shape[-1], output_depth],initializer=tf.truncated_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    net = tf.nn.conv2d(net, weights, strides=[1]+stride+[1], padding='SAME')
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def linear(net,wscope,bscope,output_depth):
    shape = net.get_shape()       
    weights = tf.get_variable(wscope, [shape[-1], output_depth], tf.float32,tf.random_normal_initializer(stddev=0.02))
    biases = tf.get_variable(bscope, [output_depth], initializer=tf.constant_initializer(0.0))
    out_logit = tf.matmul(net, weights) + biases
    return out_logit

def deconv(net,wscope,bscope,output_shape,receptive_field=[5,5],stride=[2,2]):
    weights = tf.get_variable(wscope, receptive_field+[output_shape[-1], net.get_shape()[-1]],
                                       initializer=tf.random_normal_initializer(stddev=0.02))
    net = tf.nn.conv2d_transpose(net, weights, output_shape=output_shape, strides=[1]+stride+[1])
    biases = tf.get_variable(bscope, [output_shape[-1]], initializer=tf.constant_initializer(0.0))
    net = tf.reshape(tf.nn.bias_add(net, biases), net.get_shape())
    return net

def get_shape(net):
    return net.get_shape().as_list()

# borrowed from https://github.com/shekkizh/EBGAN.tensorflow/blob/master/EBGAN/Faces_EBGAN.py
def pullaway_loss(embeddings):
    """
    Pull Away loss calculation
    :param embeddings: The embeddings to be orthogonalized for varied faces. Shape [batch_size, embeddings_dim]
    :return: pull away term loss
    """
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    similarity = tf.matmul(normalized_embeddings, normalized_embeddings, transpose_b=True)
    batch_size = tf.cast(tf.shape(embeddings)[0], tf.float32)
    pt_loss = (tf.reduce_sum(similarity) - batch_size) / (batch_size * (batch_size - 1))
    return pt_loss

# Discriminator Network

Here,i will write two "discriminator" function, the "discriminator_y" only has more one "y" parameter than "discriminator". Spliting them into two function in order to easily understanding the structure

In [4]:
def discriminator(inputs, batch_size, output_height=28, output_width=28,output_depth=1,is_training=True, reuse=False):
    # It must be Auto-Encoder style architecture
    # Architecture : (64)4c2s-FC32-FC64*14*14_BR-(1)4dc2s_S    
    with tf.variable_scope("discriminator", reuse=reuse):
        
        '''1st: Conv -> relu'''
        net = conv(inputs,'d_wconv1','d_bconv1',output_depth=64,receptive_field=[4,4])
        net = tf.nn.relu(net)

        '''2nd: reshape -> linear'''
        net = tf.reshape(net,[batch_size,-1])
        code = linear(net,'d_wlinear2','d_blinear2',32)

        '''3th: linear -> bn -> relu'''
        net = linear(code,'d_wlinear3','d_blinear3',64*14*14)
        net = bn(net, scope = 'd_bn3',is_training=is_training)
        net = tf.nn.relu(net)

        '''4th: reshape -> deconv -> sigmoid'''
        net = tf.reshape(net,[batch_size,14,14,64])
        output_shape = [batch_size, output_height, output_width, output_depth]
        net = deconv(net,'d_wdeconv4','d_bdeconv4',output_shape, receptive_field=[4,4])
        out = tf.nn.sigmoid(net)

        #recon loss
        recon_error = tf.sqrt(2 * tf.nn.l2_loss(out - inputs)) / batch_size

        return out, recon_error, code

# Generator Network

Here,i will write two "generator" function, the "generator_y" only has more one "y" parameter than "generator". Spliting them into two function in order to easily understanding the structure

In [5]:
def generator( z, batch_size, output_height=28, output_width=28,output_depth=1, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S    
    height0,width0 = output_height,output_width
    height2,width2 = math.ceil(float(height0)/2),math.ceil(float(width0)/2)
    height4,width4 = math.ceil(float(height2)/2),math.ceil(float(width2)/2)
    height8,width8 = math.ceil(float(height4)/2),math.ceil(float(width4)/2)
    height16,width16 = math.ceil(float(height8)/2),math.ceil(float(width8)/2)
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''1st: linear -> bn -> relu '''
        net = linear(z,"g_wlinear1","g_blinear1",1024)
        net = bn(net,scope='g_bn1',is_training=is_training)
        net = tf.nn.relu(net)  
    
        '''2nd: linear -> bn -> relu '''
        net = linear(net,'g_wlinear2','g_blinear2',128*7*7)
        net = bn(net,scope='g_bn2',is_training=is_training)
        net = tf.nn.relu(net)

        '''3th: reshape -> deconv -> bn -> relu '''
        net = tf.reshape(net, [batch_size, 7, 7, 128])
        output_shape = [batch_size, 14, 14, 64]
        net = deconv(net,'g_wdeconv3','g_bdeconv3',output_shape, receptive_field=[4,4])
        net = bn(net,scope='g_bn3',is_training=is_training)
        net = tf.nn.relu(net)

        '''4th: deconv -> sigmoid '''
        output_shape = [batch_size, output_height, output_width, output_depth]
        net = deconv(net,'g_wdeconv4','g_bdeconv4',output_shape,receptive_field=[4,4])
        out = tf.nn.sigmoid(net)

        return out

# set the global parameters.

In [6]:
# some parameters
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 100
y_dim = 10
learning_rate = 0.0002
beta1 = 0.5
pt_loss_weight = 0.1
# usually margin of 1 is enough, but for large batch size it must be larger than 1
margin = max(1,batch_size/64.)
output_height,output_width,output_depth = [28,28,1]
""" Graph Input """
# images
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
#labels
#y = tf.placeholder(tf.float32, [batch_size,y_dim], name='y')
# noises
z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z')

# Loss Function 

In [7]:
""" Loss Function """

# output of D for real images
D_real, D_real_err, D_real_code = discriminator(inputs, batch_size, is_training=True, reuse=False)
# output of D for fake images
G = generator(z, batch_size, is_training=True, reuse=False)
D_fake, D_fake_err, D_fake_code = discriminator(G, batch_size, is_training=True, reuse=True)

    
# get loss for discriminator
#d_loss_real = tf.reduce_mean(
#              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
#d_loss_fake = tf.reduce_mean(
#              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
#d_loss = d_loss_real + d_loss_fake
d_loss = D_real_err + tf.maximum(margin-D_fake_err, 0)



# get loss for generator
#g_loss = tf.reduce_mean(
#         tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))
g_loss = D_fake_err + pt_loss_weight*pullaway_loss(D_fake_code)

split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [8]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate*5, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
fake_images = generator(z,batch_size, is_training=False, reuse=True)

# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim))
test_labels_onehot = np.zeros([batch_size, y_dim],dtype = np.float32)
test_labels_onehot[np.arange(batch_size), np.arange(batch_size)%int(np.sqrt(batch_size))] = 1.0

# Training a GAN

In [10]:
start_epoch = 0
start_batch_id = 0


num_steps = 6000
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images,batch_labels = real_image_batch
    batch_images = batch_images.reshape([batch_size,28,28,1]).astype(np.float32)
    batch_labels = real_image_batch[1].astype(np.float32)
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)


    # update D network
    _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, z: batch_z})
    # update G network
    _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})
    _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})

        
    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(step_ind, 300) == 0:

        samples = sess.run(fake_images, feed_dict={z: sample_z})
        # put the "batch_size" images into one big canvas
        row = col = int(np.sqrt(batch_size))
        img = np.zeros( [row*28, col*28] )
        for i in range(row):
            for j in range(col):
                img[i*28:(i+1)*28,j*28:(j+1)*28] = samples[i*col+j, :, :, :].squeeze()
        #save the result      
        scipy.misc.imsave('{}.jpg'.format(step_ind),img)

Step: [0] d_loss: 2.45882535, g_loss: 0.34812006
Step: [1] d_loss: 2.53511095, g_loss: 0.21076630
Step: [2] d_loss: 2.56526852, g_loss: 0.18194437
Step: [3] d_loss: 2.54498911, g_loss: 0.16014083
Step: [4] d_loss: 2.57091236, g_loss: 0.15100284
Step: [5] d_loss: 2.56231451, g_loss: 0.14720303
Step: [6] d_loss: 2.54625654, g_loss: 0.14526439
Step: [7] d_loss: 2.54714274, g_loss: 0.14290920
Step: [8] d_loss: 2.53388119, g_loss: 0.14021786
Step: [9] d_loss: 2.52435565, g_loss: 0.13336349
Step: [10] d_loss: 2.52245092, g_loss: 0.13507384
Step: [11] d_loss: 2.49683666, g_loss: 0.13153760
Step: [12] d_loss: 2.48992157, g_loss: 0.13418341
Step: [13] d_loss: 2.46706533, g_loss: 0.12897038
Step: [14] d_loss: 2.46490669, g_loss: 0.13702089
Step: [15] d_loss: 2.43763924, g_loss: 0.12864146
Step: [16] d_loss: 2.43481350, g_loss: 0.13071549
Step: [17] d_loss: 2.42003536, g_loss: 0.13110904
Step: [18] d_loss: 2.40537620, g_loss: 0.15050592
Step: [19] d_loss: 2.38636112, g_loss: 0.15532562
Step: [20]

Step: [164] d_loss: 1.32165456, g_loss: 0.54878497
Step: [165] d_loss: 1.25413704, g_loss: 0.48045337
Step: [166] d_loss: 1.34613323, g_loss: 0.51363897
Step: [167] d_loss: 1.32709050, g_loss: 0.47334892
Step: [168] d_loss: 1.38996387, g_loss: 0.45970866
Step: [169] d_loss: 1.29151344, g_loss: 0.54355234
Step: [170] d_loss: 1.11024451, g_loss: 0.54805684
Step: [171] d_loss: 1.19837451, g_loss: 0.52248418
Step: [172] d_loss: 1.30640519, g_loss: 0.51799989
Step: [173] d_loss: 1.35418868, g_loss: 0.59663481
Step: [174] d_loss: 1.33863974, g_loss: 0.55863345
Step: [175] d_loss: 1.33385777, g_loss: 0.48582935
Step: [176] d_loss: 1.23627281, g_loss: 0.48883051
Step: [177] d_loss: 1.09822226, g_loss: 0.59771180
Step: [178] d_loss: 1.26156020, g_loss: 0.61711007
Step: [179] d_loss: 1.17876911, g_loss: 0.52101731
Step: [180] d_loss: 1.28948784, g_loss: 0.51226658
Step: [181] d_loss: 1.20011377, g_loss: 0.63708514
Step: [182] d_loss: 1.38231683, g_loss: 0.48766226
Step: [183] d_loss: 1.32755280,

Step: [326] d_loss: 1.06667662, g_loss: 0.56945294
Step: [327] d_loss: 1.10414577, g_loss: 0.60959315
Step: [328] d_loss: 0.85938483, g_loss: 0.57110018
Step: [329] d_loss: 0.92329788, g_loss: 0.57803214
Step: [330] d_loss: 1.02195740, g_loss: 0.63913214
Step: [331] d_loss: 1.11049843, g_loss: 0.57073140
Step: [332] d_loss: 0.98846269, g_loss: 0.62286174
Step: [333] d_loss: 0.67113847, g_loss: 0.60892993
Step: [334] d_loss: 1.18679309, g_loss: 0.53126270
Step: [335] d_loss: 1.05282307, g_loss: 0.59312737
Step: [336] d_loss: 1.10918164, g_loss: 0.55066556
Step: [337] d_loss: 1.22292197, g_loss: 0.44300890
Step: [338] d_loss: 1.18808103, g_loss: 0.50395459
Step: [339] d_loss: 1.19304466, g_loss: 0.54529923
Step: [340] d_loss: 1.13306189, g_loss: 0.48967084
Step: [341] d_loss: 1.06111836, g_loss: 0.63746172
Step: [342] d_loss: 0.74199414, g_loss: 0.83928913
Step: [343] d_loss: 0.76327306, g_loss: 0.82171708
Step: [344] d_loss: 1.03622353, g_loss: 0.66235054
Step: [345] d_loss: 0.99033320,

Step: [488] d_loss: 0.92878717, g_loss: 0.61327732
Step: [489] d_loss: 0.94215906, g_loss: 0.64717281
Step: [490] d_loss: 0.98465264, g_loss: 0.54440641
Step: [491] d_loss: 0.66394049, g_loss: 0.87985122
Step: [492] d_loss: 0.54623944, g_loss: 0.78626114
Step: [493] d_loss: 0.75887632, g_loss: 0.79782242
Step: [494] d_loss: 0.75299889, g_loss: 0.69951588
Step: [495] d_loss: 0.83094758, g_loss: 0.75469899
Step: [496] d_loss: 0.88223451, g_loss: 0.70271993
Step: [497] d_loss: 0.92009383, g_loss: 0.58017606
Step: [498] d_loss: 0.76646417, g_loss: 0.73145759
Step: [499] d_loss: 0.88301301, g_loss: 0.63161778
Step: [500] d_loss: 0.52840722, g_loss: 0.58522695
Step: [501] d_loss: 0.92561769, g_loss: 0.60091770
Step: [502] d_loss: 0.65387774, g_loss: 0.64896905
Step: [503] d_loss: 0.89896357, g_loss: 0.66665035
Step: [504] d_loss: 0.89427221, g_loss: 0.64321893
Step: [505] d_loss: 0.73007458, g_loss: 0.73001945
Step: [506] d_loss: 0.79878169, g_loss: 0.69900584
Step: [507] d_loss: 0.85939068,

Step: [650] d_loss: 0.67782640, g_loss: 0.75032741
Step: [651] d_loss: 0.69665504, g_loss: 0.78109568
Step: [652] d_loss: 0.48471510, g_loss: 0.94889641
Step: [653] d_loss: 0.54092014, g_loss: 0.95462883
Step: [654] d_loss: 0.41437969, g_loss: 0.94452631
Step: [655] d_loss: 0.42846659, g_loss: 0.92819536
Step: [656] d_loss: 0.58256078, g_loss: 0.85913754
Step: [657] d_loss: 0.48061880, g_loss: 0.98623461
Step: [658] d_loss: 0.62535429, g_loss: 0.85466260
Step: [659] d_loss: 0.62304068, g_loss: 0.87828386
Step: [660] d_loss: 0.47891578, g_loss: 0.85174108
Step: [661] d_loss: 0.49367005, g_loss: 0.88079011
Step: [662] d_loss: 0.57273805, g_loss: 0.90715814
Step: [663] d_loss: 0.45724261, g_loss: 0.93516135
Step: [664] d_loss: 0.51604736, g_loss: 0.83874398
Step: [665] d_loss: 0.61930645, g_loss: 0.85830724
Step: [666] d_loss: 0.63294113, g_loss: 0.85836500
Step: [667] d_loss: 0.58371520, g_loss: 0.81017780
Step: [668] d_loss: 0.70406497, g_loss: 0.85445255
Step: [669] d_loss: 0.62872088,

...

Step: [5267] d_loss: 0.64435351, g_loss: 0.87795103
Step: [5268] d_loss: 0.78365469, g_loss: 0.90065444
Step: [5269] d_loss: 0.74956846, g_loss: 0.91974896
Step: [5270] d_loss: 0.86951995, g_loss: 0.88622469
Step: [5271] d_loss: 0.68280661, g_loss: 0.99341059
Step: [5272] d_loss: 0.68136275, g_loss: 0.93058753
Step: [5273] d_loss: 0.67559338, g_loss: 0.94021797
Step: [5274] d_loss: 0.72544867, g_loss: 0.85007674
Step: [5275] d_loss: 0.67131525, g_loss: 0.89484787
Step: [5276] d_loss: 0.67593914, g_loss: 0.86615306
Step: [5277] d_loss: 0.64133906, g_loss: 0.91427821
Step: [5278] d_loss: 0.81386381, g_loss: 0.81361955
Step: [5279] d_loss: 0.69128883, g_loss: 0.89594382
Step: [5280] d_loss: 0.68607199, g_loss: 0.92616469
Step: [5281] d_loss: 0.70521718, g_loss: 0.94075286
Step: [5282] d_loss: 0.77960849, g_loss: 0.94177794
Step: [5283] d_loss: 0.77379727, g_loss: 0.98102260
Step: [5284] d_loss: 0.67967659, g_loss: 0.91294146
Step: [5285] d_loss: 0.70396990, g_loss: 0.93818706
Step: [5286]

Step: [5426] d_loss: 0.62062955, g_loss: 0.92732066
Step: [5427] d_loss: 0.73386467, g_loss: 0.90998977
Step: [5428] d_loss: 0.82163626, g_loss: 0.84902966
Step: [5429] d_loss: 0.70247716, g_loss: 0.97778547
Step: [5430] d_loss: 0.75767720, g_loss: 0.91527009
Step: [5431] d_loss: 0.77184951, g_loss: 0.90507489
Step: [5432] d_loss: 0.68215388, g_loss: 0.88882166
Step: [5433] d_loss: 0.82773972, g_loss: 0.86317325
Step: [5434] d_loss: 0.73639119, g_loss: 0.86814010
Step: [5435] d_loss: 0.67215055, g_loss: 0.95927787
Step: [5436] d_loss: 0.73303735, g_loss: 0.86586571
Step: [5437] d_loss: 0.77336961, g_loss: 0.92655724
Step: [5438] d_loss: 0.70546830, g_loss: 0.89842671
Step: [5439] d_loss: 0.66319388, g_loss: 0.90893221
Step: [5440] d_loss: 0.78954315, g_loss: 0.85349524
Step: [5441] d_loss: 0.77690792, g_loss: 0.85610002
Step: [5442] d_loss: 0.78900325, g_loss: 0.91655254
Step: [5443] d_loss: 0.73975015, g_loss: 0.92092454
Step: [5444] d_loss: 0.61956602, g_loss: 0.90402204
Step: [5445]

Step: [5585] d_loss: 0.73335314, g_loss: 0.89390892
Step: [5586] d_loss: 0.70402569, g_loss: 0.93097305
Step: [5587] d_loss: 0.78082925, g_loss: 0.94712144
Step: [5588] d_loss: 0.78993893, g_loss: 1.00129223
Step: [5589] d_loss: 0.68123698, g_loss: 0.91646284
Step: [5590] d_loss: 0.68638539, g_loss: 0.96543467
Step: [5591] d_loss: 0.86959928, g_loss: 0.98667848
Step: [5592] d_loss: 0.82237405, g_loss: 0.82199824
Step: [5593] d_loss: 0.69334137, g_loss: 0.91652256
Step: [5594] d_loss: 0.79702520, g_loss: 0.93714416
Step: [5595] d_loss: 0.65028137, g_loss: 0.93197304
Step: [5596] d_loss: 0.70031548, g_loss: 0.94632506
Step: [5597] d_loss: 0.68396324, g_loss: 0.92932630
Step: [5598] d_loss: 0.68886721, g_loss: 0.92199361
Step: [5599] d_loss: 0.65883714, g_loss: 0.95463502
Step: [5600] d_loss: 0.63188285, g_loss: 0.90263307
Step: [5601] d_loss: 0.82620895, g_loss: 0.86332428
Step: [5602] d_loss: 0.76320398, g_loss: 0.91133159
Step: [5603] d_loss: 0.75214458, g_loss: 0.95050126
Step: [5604]

Step: [5744] d_loss: 0.74339437, g_loss: 0.88653541
Step: [5745] d_loss: 0.73872191, g_loss: 0.96176159
Step: [5746] d_loss: 0.83082032, g_loss: 0.89169085
Step: [5747] d_loss: 0.73774499, g_loss: 0.91398799
Step: [5748] d_loss: 0.72161412, g_loss: 0.94051075
Step: [5749] d_loss: 0.82843709, g_loss: 0.92319834
Step: [5750] d_loss: 0.66182429, g_loss: 0.92389840
Step: [5751] d_loss: 0.66818804, g_loss: 0.93274152
Step: [5752] d_loss: 0.66671073, g_loss: 0.96762019
Step: [5753] d_loss: 0.78590870, g_loss: 0.95235556
Step: [5754] d_loss: 0.71090174, g_loss: 0.92741048
Step: [5755] d_loss: 0.76427376, g_loss: 0.93253857
Step: [5756] d_loss: 0.70128834, g_loss: 0.98373818
Step: [5757] d_loss: 0.66803515, g_loss: 0.90908432
Step: [5758] d_loss: 0.71933031, g_loss: 0.91850823
Step: [5759] d_loss: 0.73574609, g_loss: 0.96253484
Step: [5760] d_loss: 0.72086024, g_loss: 0.90732205
Step: [5761] d_loss: 0.67352670, g_loss: 0.91095281
Step: [5762] d_loss: 0.73459518, g_loss: 0.94478083
Step: [5763]

Step: [5904] d_loss: 0.65445393, g_loss: 0.95218873
Step: [5905] d_loss: 0.74600160, g_loss: 0.85066628
Step: [5906] d_loss: 0.73592281, g_loss: 0.93025446
Step: [5907] d_loss: 0.79711652, g_loss: 0.87063938
Step: [5908] d_loss: 0.75547570, g_loss: 0.90161693
Step: [5909] d_loss: 0.63561666, g_loss: 0.91892481
Step: [5910] d_loss: 0.68241507, g_loss: 0.93961447
Step: [5911] d_loss: 0.72997171, g_loss: 0.89404911
Step: [5912] d_loss: 0.65236926, g_loss: 0.92904431
Step: [5913] d_loss: 0.66824418, g_loss: 0.92212659
Step: [5914] d_loss: 0.63888931, g_loss: 0.94057935
Step: [5915] d_loss: 0.63401592, g_loss: 0.92658132
Step: [5916] d_loss: 0.74660778, g_loss: 0.88201892
Step: [5917] d_loss: 0.74404502, g_loss: 0.94818068
Step: [5918] d_loss: 0.67099166, g_loss: 0.99898636
Step: [5919] d_loss: 0.71599197, g_loss: 0.99632573
Step: [5920] d_loss: 0.65042937, g_loss: 0.93511456
Step: [5921] d_loss: 0.78414589, g_loss: 0.88675874
Step: [5922] d_loss: 0.66831881, g_loss: 0.86513549
Step: [5923]